In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Err:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Release
  404  Not Found [IP: 65.8.185.33 443]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-28 23:58:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.72MB/s    in 0.2s    

2021-09-28 23:58:41 (5.72 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-01-31 00:08:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [5]:
  # review_id TEXT PRIMARY KEY NOT NULL,
  # customer_id INTEGER,
  # product_id TEXT,
  # product_parent INTEGER,
  # review_date DATE -- this should be in the formate yyyy-mm-dd

In [6]:
filtered_df=df.select(["review_id","customer_id","product_id","product_parent","review_date"])
filtered_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781|2015-01-31 00:08:00|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853|2015-01-31 00:08:00|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660|2015-01-31 00:08:00|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263|2015-01-31 00:08:00|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171|2015-01-31 00:08:00|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457|2015-01-31 00:08:00|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602|2015-01-31 00:08:00|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055|2015-01-31 00:08:00|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438|2015-01-31 00:08:00|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213|2015-01-31 00:08:00|
|R369CEXHXHC6NQ|   134094

## Examine the schema

In [7]:
filtered_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Write DataFrame to RDS

In [8]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://postgres.cqedskjvnup6.us-east-2.rds.amazonaws.com:5432/big_data_hw"

config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [9]:
# Write DataFrame to table

filtered_df.write.jdbc(url=jdbc_url, table='review_shoes_table', mode=mode, properties=config)